In [1]:
!pip install rouge-score

import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from tqdm.notebook import tqdm
import os

# =====================================================
# CONFIG
# =====================================================
MAX_DOC_ID = 2960
CHUNK_SIZE = 50

# =====================================================
# LOAD DATA
# =====================================================
print("Loading data...")
extractive_df = pd.read_csv('/kaggle/input/legal-document-summary-dataset/extracted_summaries_with_eval.csv')
train_df = pd.read_csv('/kaggle/input/legal-document-summary-dataset/train_data.csv')

extractive_df = extractive_df[extractive_df['doc_id'] <= MAX_DOC_ID]
train_df = train_df.iloc[:MAX_DOC_ID + 1]

extractive_by_doc = extractive_df.groupby('doc_id')

# =====================================================
# INITIALIZE ROUGE SCORER (FASTER!)
# =====================================================
print("Initializing ROUGE scorer...")
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
print("✅ ROUGE scorer loaded successfully!")

# =====================================================
# IMPROVED CHECKPOINT MANAGEMENT
# =====================================================
def save_checkpoint(doc_id, samples, chunk_number):
    """Save checkpoint data with error handling"""
    checkpoint_file = f'/kaggle/working/checkpoint_doc_{doc_id}chunk{chunk_number}.csv'
    try:
        if samples:  # Only save if we have samples
            df = pd.DataFrame(samples)
            df.to_csv(checkpoint_file, index=False)
            print(f"💾 Checkpoint saved: {checkpoint_file} ({len(samples)} samples)")
        else:
            print(f"⚠  No samples to save for chunk {chunk_number}")
    except Exception as e:
        print(f"❌ Error saving checkpoint {checkpoint_file}: {e}")

def load_checkpoints():
    """Load existing checkpoints with better error handling"""
    checkpoint_files = [f for f in os.listdir('/kaggle/working') 
                       if f.startswith('checkpoint_doc_') and f.endswith('.csv')]
    
    if not checkpoint_files:
        print("🆕 No valid checkpoints found, starting fresh")
        return [], 0
    
    all_samples = []
    max_doc_id = 0
    valid_checkpoints = 0
    
    for file in sorted(checkpoint_files):
        file_path = f'/kaggle/working/{file}'
        try:
            # Check if file is not empty
            if os.path.getsize(file_path) == 0:
                print(f"⚠  Skipping empty file: {file}")
                continue
                
            df = pd.read_csv(file_path)
            if len(df) > 0:
                all_samples.extend(df.to_dict('records'))
                # Extract max doc_id from filename
                doc_id = int(file.split('_')[2])
                max_doc_id = max(max_doc_id, doc_id)
                valid_checkpoints += 1
                print(f"📂 Loaded checkpoint: {file} ({len(df)} samples, doc_id: {doc_id})")
            else:
                print(f"⚠  Empty dataframe in: {file}")
                
        except Exception as e:
            print(f"❌ Error loading checkpoint {file}: {e}")
    
    if valid_checkpoints > 0:
        resume_from = max_doc_id + 1
        print(f"🔄 Resuming from doc_id {resume_from} ({valid_checkpoints} valid checkpoints loaded)")
    else:
        resume_from = 0
        print("🚀 No valid checkpoints, starting fresh")
        
    return all_samples, resume_from

def clean_corrupted_checkpoints():
    """Remove corrupted/empty checkpoint files"""
    checkpoint_files = [f for f in os.listdir('/kaggle/working') 
                       if f.startswith('checkpoint_doc_') and f.endswith('.csv')]
    
    cleaned_count = 0
    for file in checkpoint_files:
        file_path = f'/kaggle/working/{file}'
        try:
            # Remove empty files
            if os.path.getsize(file_path) == 0:
                os.remove(file_path)
                cleaned_count += 1
                print(f"🧹 Removed empty file: {file}")
            # Try to read the file to check if it's corrupted
            else:
                pd.read_csv(file_path)
        except:
            # File is corrupted, remove it
            os.remove(file_path)
            cleaned_count += 1
            print(f"🧹 Removed corrupted file: {file}")
    
    if cleaned_count > 0:
        print(f"🧹 Cleaned {cleaned_count} corrupted/empty checkpoint files")

# =====================================================
# CLEAN UP CORRUPTED CHECKPOINTS FIRST
# =====================================================
print("🧹 Checking for corrupted checkpoints...")
clean_corrupted_checkpoints()

# =====================================================
# HELPER FUNCTIONS - ROUGE VERSION
# =====================================================

def prepare_data(doc_id):
    """Prepare data for a single document"""
    try:
        doc_extractive = extractive_by_doc.get_group(doc_id)
        gt_summary = train_df.iloc[doc_id]['summary']
        
        if pd.isna(gt_summary) or str(gt_summary).strip() == "":
            return None, None
            
        gt_sentences = sent_tokenize(gt_summary)

        extractive_summaries = {}
        for _, row in doc_extractive.iterrows():
            method = row['method']
            text = row['extractive_doc']
            
            if pd.isna(text) or str(text).strip() == "":
                continue
                
            extractive_summaries[method] = {
                'text': text,
                'sentences': sent_tokenize(text)
            }

        return gt_sentences, extractive_summaries
    except Exception as e:
        print(f"❌ Error preparing data for doc_id {doc_id}: {e}")
        return None, None

def calculate_rouge_similarity(gt_sentence, extractive_sentences):
    """Compute average ROUGE similarity (FASTER!)"""
    if not extractive_sentences:
        return 0
    
    try:
        similarities = []
        for ext_sentence in extractive_sentences:
            scores = rouge_scorer_obj.score(gt_sentence, ext_sentence)
            # Average of ROUGE-1, ROUGE-2, ROUGE-L F1 scores (as in paper)
            avg_f1 = (scores['rouge1'].fmeasure + 
                     scores['rouge2'].fmeasure + 
                     scores['rougeL'].fmeasure) / 3
            similarities.append(avg_f1)
        return np.mean(similarities) if similarities else 0
    except Exception as e:
        print(f"❌ ROUGE calculation error: {e}")
        return 0

def assign_sentences_to_extractive(gt_sentences, extractive_summaries):
    """Assign GT sentences to the most similar extractive summaries"""
    if not extractive_summaries:
        return {}
        
    assigned = {m: [] for m in extractive_summaries.keys()}

    for gt_sentence in gt_sentences:
        scores = {}
        for method, data in extractive_summaries.items():
            score = calculate_rouge_similarity(gt_sentence, data['sentences'])
            scores[method] = score

        sorted_methods = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        if not sorted_methods:
            continue
            
        top_method = sorted_methods[0][0]
        second_method = sorted_methods[1][0] if len(sorted_methods) > 1 else top_method

        current_assigned = ' '.join(assigned[top_method] + [gt_sentence])
        ext_doc_len = len(extractive_summaries[top_method]['text'].split())
        if len(current_assigned.split()) <= (0.5 * ext_doc_len):
            assigned[top_method].append(gt_sentence)
        else:
            assigned[second_method].append(gt_sentence)
    return assigned

def generate_training_samples(doc_id, assigned, extractive_summaries):
    """Generate final training samples for a document"""
    samples = []
    for method, gt_sentences in assigned.items():
        if not gt_sentences:
            continue
        new_summary = ' '.join(gt_sentences)
        extractive_doc = extractive_summaries[method]['text']
        samples.append({
            'doc_id': doc_id,
            'method': method,
            'document': extractive_doc,
            'summary': new_summary,
            'original_doc_length': len(extractive_doc.split()),
            'summary_length': len(new_summary.split())
        })
    return samples

# =====================================================
# MAIN FUNCTION
# =====================================================
def build_eta_dataset_with_checkpoints():
    # Load existing checkpoints
    all_samples, start_doc_id = load_checkpoints()
    
    doc_ids = sorted(extractive_df['doc_id'].unique())
    doc_ids_to_process = [doc_id for doc_id in doc_ids if doc_id >= start_doc_id]
    
    print(f"📊 Processing {len(doc_ids_to_process)} documents...")
    
    with tqdm(total=len(doc_ids_to_process), desc="Building ETA dataset") as pbar:
        chunk_samples = []
        chunk_number = 0
        
        for i, doc_id in enumerate(doc_ids_to_process):
            try:
                gt_sentences, extractive_summaries = prepare_data(doc_id)
                
                if gt_sentences is not None and extractive_summaries is not None:
                    assigned = assign_sentences_to_extractive(gt_sentences, extractive_summaries)
                    samples = generate_training_samples(doc_id, assigned, extractive_summaries)
                    chunk_samples.extend(samples)
                    all_samples.extend(samples)
                
                pbar.update(1)
                pbar.set_postfix({
                    'current_doc': doc_id,
                    'total_samples': len(all_samples),
                    'chunk': chunk_number
                })
                
                # Save checkpoint every CHUNK_SIZE documents
                if (i + 1) % CHUNK_SIZE == 0 and chunk_samples:
                    chunk_number += 1
                    save_checkpoint(doc_id, chunk_samples, chunk_number)
                    chunk_samples = []
                    
            except Exception as e:
                print(f"❌ Error at doc_id {doc_id}: {e}")
                continue
    
    # Save final chunk
    if chunk_samples:
        chunk_number += 1
        save_checkpoint(doc_ids_to_process[-1], chunk_samples, chunk_number)
    
    return pd.DataFrame(all_samples)

# =====================================================
# EXECUTION
# =====================================================
print("🚀 Starting ETA dataset building with ROUGE...")
eta_dataset = build_eta_dataset_with_checkpoints()

# Save final output
final_output_file = '/kaggle/working/eta_rouge_half_dataset.csv'
eta_dataset.to_csv(final_output_file, index=False)

print(f"✅ Final dataset saved: {final_output_file}")
print(f"📊 Total samples generated: {len(eta_dataset)}")

# Clean up checkpoint files
print("🧹 Cleaning up checkpoint files...")
checkpoint_files = [f for f in os.listdir('/kaggle/working') if f.startswith('checkpoint_doc_')]
for file in checkpoint_files:
    os.remove(f'/kaggle/working/{file}')
print(f"🧹 Deleted {len(checkpoint_files)} checkpoint files")

print("🎉 ETA dataset building completed successfully!")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ae3d9a47b059d43cca870c97f0d4efd3228e5567fd5942e022b192f4aa140402
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
Loading data...
Initializing ROUGE scorer...
✅ ROUGE scorer loaded successfully!
🧹 Checking for corrupted checkpoints...
🚀 Starting ETA dataset building with ROUGE...
🆕 No valid checkpoints found, starting fresh
📊 Processing 2961 documents...


Building ETA dataset:   0%|          | 0/2961 [00:00<?, ?it/s]

💾 Checkpoint saved: /kaggle/working/checkpoint_doc_49chunk1.csv (288 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_99chunk2.csv (279 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_149chunk3.csv (279 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_199chunk4.csv (290 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_249chunk5.csv (293 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_299chunk6.csv (278 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_349chunk7.csv (295 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_399chunk8.csv (287 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_449chunk9.csv (270 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_499chunk10.csv (277 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_549chunk11.csv (277 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_599chunk12.csv (287 samples)
💾 Checkpoint saved: /kaggle/working/checkpoint_doc_